In [53]:
import pandas as pd

data = pd.read_pickle('./universal/data/nyse_o.pkl')

data = data.fillna(method='ffill')

data_r = (data / data.shift(1)).fillna(1)

In [129]:
w = np.ones(5)

In [131]:
np.sum(w*w)

5.0

In [2]:
import pickle as pkl
X = pkl.load(open("nyse_o_np.pkl", "rb"))

In [3]:
#We take only T = 500 here  for optimization
T = 500
X = X[:,:T]

In [188]:
logger_file = './'
logger = buildLogger(logger_file, 'CORN', logging.DEBUG)
P = 5
K = 5
W = 6

# Baseline

In [16]:
import sys
import numpy as np
from itertools import takewhile
import logging
from logbuilder import buildLogger
import cvxpy as cp

def searchOpt(C):
    """
    @C: historical prices with corr high than rho
    """
    numAssets = C.shape[0]
    logger = logging.getLogger('CORN') 
    
    b = cp.Variable(numAssets, pos = True)
    prob = cp.Problem(cp.Minimize(-1 * cp.sum(cp.log(C.T * b))), [cp.sum(b) == 1])
    logger.debug('Solving Problem: ' + str(prob.solve(solver = 'SCS', warm_start = True)))
    return np.array(b.value)


class expert():
    def __init__(self, rho, timeWindow, initialWealth = 1):
        """
        initialize:
        @rho: correlation threshold
        @timeWindow: specific time windows
        @initialWealth: wealth
        """
        self.rho =rho
        self.timeWindow = timeWindow
        self.wealth = initialWealth

    def __eq__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth == other.wealth

    def __lt__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth < other.wealth

    def learning(self, Xh):
        """
        @Xh: historical prices until time t, m*(t-1)
        Return:
        @portfolio: array of weights put on each assets
        """
        # Use h instead of t here since h = t-1
        numAssets, h = Xh.shape
        self.portfolio = np.ones(numAssets)/numAssets
        # if have reached w(timeWindow) + 1 days
        if h > self.timeWindow:
            indexSet = [np.corrcoef(Xh[:,-self.timeWindow:].reshape(-1), Xh[:,i-self.timeWindow: i].reshape(-1))[1][0] >= self.rho\
                        for i in range(self.timeWindow ,h)]
            if sum(indexSet) > 0:
                self.portfolio = searchOpt(Xh[:,self.timeWindow:][:,indexSet])
        return self.portfolio
    def update(self, xt):
        """
        Update at the end of the day
        @xt: prices of the current trading day
        """
        self.wealth *= np.sum(self.portfolio * xt)


def preProcess(data_path):
    dataWithWeekend = np.loadtxt(data_path, delimiter=' ')
    # Remove days when market closed
    dataWithoutWeekend = dataWithWeekend[:,dataWithWeekend[0]!= 0]
    # Start Date: the number of zeros before the first non-zero data
    startDate = [len(list(takewhile(i))) for i in dataWithoutWeekend]
    # Transform the data into relative prices compared to the day before
    # Seems that a Numba function @nb.jit could help fill nulls, not used here


def combine(portfolioSet, wealthSet, q):
    """
    Combine the experts' portfolios
    @portfolioSet: array of portfolios, W * m(numAssets)
    @wealthSet: array of experts' current wealth, W
    @q: array of probability distribution function, W
    """
    nome = np.sum(portfolioSet.T * wealthSet * q, axis = 1)
    deno = np.sum(wealthSet * q)
    return nome/deno


def checkParams(**kwargs):
    if "rho" in kwargs:
        return False
    elif not "P" in kwargs and "K" in kwargs:
        raise ValueError("Definition for both P and K are required for CORNK")
    return True


def CORN(X, W, **kwargs):
    """
    main algorithm
    @W: maximum time window length
    @X: historical prices matrix, m(numAssets) * T
    There are two set of possible optional parameters:
    CORNU:
    @rho: correlation threshold
    CORNK:
    @P: max number of correlation thresholds
    @K: the value of K for top-K
    """
    logger = logging.getLogger('CORN')
    IS_CORNK = checkParams(**kwargs)
    # Initialize weights q and wealth
    wealth = 1
    wealthRecord = [1]
    numAssets, T = X.shape
    if IS_CORNK:
        K = kwargs["K"]
        P = kwargs["P"]
        q = np.ones(W*P)/(W*P)
        rhoSet = np.arange(P)/P
        expertPort = [expert(rho,w) for w in range(1,W+1) for rho in rhoSet]
    else:
        rho = kwargs["rho"]
        q = np.ones(W)/W
        expertPort = [expert(rho,w) for w in range(1,W+1)]

    for t in range(T):
        if t%100 == 1:
            print(t)
        logger.info('Start Day ' + str(t))
        portfolioSet = [expertI.learning(X[:,:t]) for expertI in expertPort]
        wealthSet = [expertI.wealth for expertI in expertPort]
        portfolioOverall = combine(np.array(portfolioSet), np.array(wealthSet), q)
        wealth *= np.sum(portfolioOverall * X[:,t])
        for expertI in expertPort:
            expertI.update(X[:,t])
        logger.info('End Day with wealth '+ str(wealth))
        wealthRecord.append(wealth)
        if IS_CORNK:
            topK = np.argsort(expertPort)[:K]
            logger.debug("The top K experts are: " +  str(topK))
            q.fill(0)
            q[topK] = 1/K
    return wealthRecord

In [17]:
start = time.time()
CORN(X, 5, rho = 0.4)
end = time.time()
print(end-start)

1


C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\numpy\lib\function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\numpy\lib\function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


101
201
301
401
82.21837782859802


# Line Profiling

In [36]:
%reload_ext line_profiler

%lprun -f CORN wealthRecord = CORN(X, W, P = P, K = K)

1
101
201
301
401


In [ ]:
Timer unit: 5.68889e-07 s

Total time: 1051.4 s
File: <ipython-input-31-3212a64250d7>
Function: CORN at line 99

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    99                                           def CORN(X, W, **kwargs):
   100                                               """
   101                                               main algorithm
   102                                               @W: maximum time window length
   103                                               @X: historical prices matrix, m(numAssets) * T
   104                                               There are two set of possible optional parameters:
   105                                               CORNU:
   106                                               @rho: correlation threshold
   107                                               CORNK:
   108                                               @P: max number of correlation thresholds
   109                                               @K: the value of K for top-K
   110                                               """
   111         1         40.0     40.0      0.0      logger = logging.getLogger('CORN')
   112         1         10.0     10.0      0.0      IS_CORNK = checkParams(**kwargs)
   113                                               # Initialize weights q and wealth
   114         1          5.0      5.0      0.0      wealth = 1
   115         1          4.0      4.0      0.0      wealthRecord = [1]
   116         1          8.0      8.0      0.0      numAssets, T = X.shape
   117         1          4.0      4.0      0.0      if IS_CORNK:
   118         1          5.0      5.0      0.0          K = kwargs["K"]
   119         1          5.0      5.0      0.0          P = kwargs["P"]
   120         1         74.0     74.0      0.0          q = np.ones(W*P)/(W*P)
   121         1         33.0     33.0      0.0          rhoSet = np.arange(P)/P
   122         1        186.0    186.0      0.0          expertPort = [expert(rho,w) for w in range(1,W+1) for rho in rhoSet]
   123                                               else:
   124                                                   q = np.ones(W)/W
   125                                                   expertPort = [expert(rho,w) for w in range(1,W+1)]
   126                                           
   127       501       1394.0      2.8      0.0      for t in range(T):
   128       500       1413.0      2.8      0.0          if t%100 == 1:
   129         5       4644.0    928.8      0.0              print(t)
   130       500     178780.0    357.6      0.0          logger.info('Start Day ' + str(t))
   131       500 1846713390.0 3693426.8     99.9          portfolioSet = [expertI.learning(X[:,:t]) for expertI in expertPort]
   132       500      20271.0     40.5      0.0          wealthSet = [expertI.wealth for expertI in expertPort]
   133       500      75004.0    150.0      0.0          portfolioOverall = combine(np.array(portfolioSet), np.array(wealthSet), q)
   134       500      15412.0     30.8      0.0          wealth *= np.sum(portfolioOverall * X[:,t])
   135     15500      35995.0      2.3      0.0          for expertI in expertPort:
   136     15000     333150.0     22.2      0.0              expertI.update(X[:,t])
   137       500     275372.0    550.7      0.0          logger.info('End Day with wealth '+ str(wealth))
   138       500       2363.0      4.7      0.0          wealthRecord.append(wealth)
   139       500       1166.0      2.3      0.0          if IS_CORNK:
   140       500     161819.0    323.6      0.0              topK = np.argsort(expertPort)[:K]
   141       500     326465.0    652.9      0.0              logger.debug("The top K experts are: " +  str(topK))
   142       500       4023.0      8.0      0.0              q.fill(0)
   143       500       3725.0      7.5      0.0              q[topK] = 1/K
   144         1          2.0      2.0      0.0      return wealthRecord

In [37]:
expertI = expert(0.4, 5)
%lprun -f expertI.learning expertI.learning(X)

In [ ]:
Timer unit: 5.68889e-07 s

Total time: 0.0772392 s
File: <ipython-input-31-3212a64250d7>
Function: learning at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    45                                               def learning(self, Xh):
    46                                                   """
    47                                                   @Xh: historical prices until time t, m*(t-1)
    48                                                   Return:
    49                                                   @portfolio: array of weights put on each assets
    50                                                   """
    51                                                   # Use h instead of t here since h = t-1
    52         1         12.0     12.0      0.0          numAssets, h = Xh.shape
    53         1        134.0    134.0      0.1          self.portfolio = np.ones(numAssets)/numAssets
    54                                                   # if have reached w(timeWindow) + 1 days
    55         1          3.0      3.0      0.0          if h > self.timeWindow:
    56         1          2.0      2.0      0.0              indexSet = [np.corrcoef(Xh[:,-self.timeWindow:].reshape(-1), Xh[:,i-self.timeWindow: i].reshape(-1))[1][0] >= self.rho\
    57         1     101891.0 101891.0     75.0                          for i in range(self.timeWindow ,h)]
    58         1       1744.0   1744.0      1.3              if sum(indexSet) > 0:
    59         1      31983.0  31983.0     23.6                  self.portfolio = searchOpt(Xh[:,self.timeWindow:][:,indexSet])
    60         1          3.0      3.0      0.0          return self.portfolio

In [40]:
def CORN_Fake(X, W, **kwargs):
    """
    main algorithm
    @W: maximum time window length
    @X: historical prices matrix, m(numAssets) * T
    There are two set of possible optional parameters:
    CORNU:
    @rho: correlation threshold
    CORNK:
    @P: max number of correlation thresholds
    @K: the value of K for top-K
    """
    logger = logging.getLogger('CORN')
    IS_CORNK = checkParams(**kwargs)
    # Initialize weights q and wealth
    wealth = 1
    wealthRecord = [1]
    numAssets, T = X.shape
    if IS_CORNK:
        K = kwargs["K"]
        P = kwargs["P"]
        q = np.ones(W*P)/(W*P)
        rhoSet = np.arange(P)/P
        expertPort = [expert(rho,w) for w in range(1,W+1) for rho in rhoSet]
    else:
        rho = kwargs["rho"]
        q = np.ones(W)/W
        expertPort = [expert(rho,w) for w in range(1,W+1)]

    for t in range(T):
        if t%100 == 1:
            print(t)
        logger.info('Start Day ' + str(t))
        portfolioSet = [np.random.rand(numAssets) for expertI in expertPort] #fake line
        wealthSet = [expertI.wealth for expertI in expertPort]
        portfolioOverall = combine(np.array(portfolioSet), np.array(wealthSet), q)
        wealth *= np.sum(portfolioOverall * X[:,t])
        for expertI in expertPort:
            expertI.portfolio = np.random.rand(numAssets)
            expertI.update(X[:,t])
        logger.info('End Day with wealth '+ str(wealth))
        wealthRecord.append(wealth)
        if IS_CORNK:
            topK = np.argsort(expertPort)[:K]
            logger.debug("The top K experts are: " +  str(topK))
            q.fill(0)
            q[topK] = 1/K
    return wealthRecord

In [42]:
%reload_ext line_profiler

%lprun -f CORN_Fake wealthRecord = CORN_Fake(X, W, P = P, K = K)

1
101
201
301


C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: overflow encountered in double_scalars
C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in multiply
C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in multiply


401


# Cython

In [43]:
%load_ext Cython

In [57]:
%%cython --annotate
import sys
import pickle as pkl
import numpy as np
from itertools import takewhile
import logging
from logbuilder import buildLogger
import cvxpy as cp
import time

def searchOpt(C):
    """
    @C: historical prices with corr high than rho
    """
    numAssets = C.shape[0]
    logger = logging.getLogger('CORN') 
    
    b = cp.Variable(numAssets, pos = True)
    prob = cp.Problem(cp.Minimize(-1 * cp.sum(cp.log(C.T * b))), [cp.sum(b) == 1])
    logger.debug('Solving Problem: ' + str(prob.solve(solver = 'SCS', warm_start = True)))
    return np.array(b.value)


class expert():
    def __init__(self, rho, timeWindow, initialWealth = 1):
        """
        initialize:
        @rho: correlation threshold
        @timeWindow: specific time windows
        @initialWealth: wealth
        """
        self.rho =rho
        self.timeWindow = timeWindow
        self.wealth = initialWealth

    def __eq__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth == other.wealth

    def __lt__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth < other.wealth

    def learning(self, Xh):
        """
        @Xh: historical prices until time t, m*(t-1)
        Return:
        @portfolio: array of weights put on each assets
        """
        # Use h instead of t here since h = t-1
        numAssets, h = Xh.shape
        self.portfolio = np.ones(numAssets)/numAssets
        # if have reached w(timeWindow) + 1 days
        if h > self.timeWindow:
            indexSet = [np.corrcoef(Xh[:,-self.timeWindow:].reshape(-1), Xh[:,i-self.timeWindow: i].reshape(-1))[1][0] >= self.rho\
                        for i in range(self.timeWindow ,h)]
            if sum(indexSet) > 0:
                self.portfolio = searchOpt(Xh[:,self.timeWindow:][:,indexSet])
        return self.portfolio
    def update(self, xt):
        """
        Update at the end of the day
        @xt: prices of the current trading day
        """
        self.wealth *= np.sum(self.portfolio * xt)


def preProcess(data_path):
    dataWithWeekend = np.loadtxt(data_path, delimiter=' ')
    # Remove days when market closed
    dataWithoutWeekend = dataWithWeekend[:,dataWithWeekend[0]!= 0]
    # Start Date: the number of zeros before the first non-zero data
    startDate = [len(list(takewhile(i))) for i in dataWithoutWeekend]
    # Transform the data into relative prices compared to the day before
    # Seems that a Numba function @nb.jit could help fill nulls, not used here


def combine(portfolioSet, wealthSet, q):
    """
    Combine the experts' portfolios
    @portfolioSet: array of portfolios, W * m(numAssets)
    @wealthSet: array of experts' current wealth, W
    @q: array of probability distribution function, m
    """
    nome = np.sum(portfolioSet.T * wealthSet * q, axis = 1)
    deno = np.sum(wealthSet * q)
    return nome/deno


def checkParams(**kwargs):
    if "rho" in kwargs:
        return False
    elif not "P" in kwargs and "K" in kwargs:
        raise ValueError("Definition for both P and K are required for CORNK")
    return True


def CORN(X, W, **kwargs):
    """
    main algorithm
    @W: maximum time window length
    @X: historical prices matrix, m(numAssets) * T
    There are two set of possible optional parameters:
    CORNU:
    @rho: correlation threshold
    CORNK:
    @P: max number of correlation thresholds
    @K: the value of K for top-K
    """
    logger = logging.getLogger('CORN')
    IS_CORNK = checkParams(**kwargs)
    # Initialize weights q and wealth
    wealth = 1
    wealthRecord = [1]
    numAssets, T = X.shape
    if IS_CORNK:
        K = kwargs["K"]
        P = kwargs["P"]
        q = np.ones(W*P)/(W*P)
        rhoSet = np.arange(P)/P
        expertPort = [expert(rho,w) for w in range(1,W+1) for rho in rhoSet]
    else:
        rho = kwargs["rho"]
        q = np.ones(W)/W
        expertPort = [expert(rho,w) for w in range(1,W+1)]

    for t in range(T):
        if t%100 == 1:
            print(t)
        logger.info('Start Day ' + str(t))
        portfolioSet = [expertI.learning(X[:,:t]) for expertI in expertPort]
        wealthSet = [expertI.wealth for expertI in expertPort]
        portfolioOverall = combine(np.array(portfolioSet), np.array(wealthSet), q)
        wealth *= np.sum(portfolioOverall * X[:,t])
        for expertI in expertPort:
            expertI.update(X[:,t])
        logger.info('End Day with wealth '+ str(wealth))
        wealthRecord.append(wealth)
        if IS_CORNK:
            topK = np.argsort(expertPort)[:K]
            logger.debug("The top K experts are: " +  str(topK))
            q.fill(0)
            q[topK] = 1/K
    return wealthRecord


start = time.time()
X = pkl.load(open("nyse_o_np.pkl", "rb"))[:,:500]
W = 6
P=5
K=5
rho = 0.4
wealthRecord = CORN(X, W, P = P, K = K)
print("total time: ", time.time() - start)

1
101
201
301
401
total time:  832.6704740524292


In [2]:
%load_ext Cython

In [4]:
%%cython --annotate
import pickle as pkl
import numpy as np
cimport numpy as np
from itertools import takewhile
import logging
from logbuilder import buildLogger
import cvxpy as cp
import time

def searchOpt(np.ndarray[float, ndim=2] C):
    """
    @C: historical prices with corr high than rho
    """
    cdef int numAssets = C.shape[0]
    logger = logging.getLogger('CORN') 
    
    b = cp.Variable(numAssets, pos = True)
    prob = cp.Problem(cp.Minimize(-1 * cp.sum(cp.log(C.T * b))), [cp.sum(b) == 1])
    logger.debug('Solving Problem: ' + str(prob.solve(solver = 'SCS', warm_start = True)))
    return np.array(b.value, dtype = np.float32)


cdef class expert():
    cdef public float wealth
    cdef float rho
    cdef int timeWindow
    cdef dict __dict__
    
    def __cinit__(self, float rho,int timeWindow,float initialWealth):
        """
        initialize:
        @rho: correlation threshold
        @timeWindow: specific time windows
        @initialWealth: wealth
        """
        self.rho =rho
        self.timeWindow = timeWindow
        self.wealth = initialWealth

    def __eq__(self, expert other):
        """
        Used for wealth ranking
        """
        return self.wealth == other.wealth

    def __lt__(self, expert other):
        """
        Used for wealth ranking
        """
        return self.wealth < other.wealth

    def learning(self, np.ndarray[float, ndim=2] Xh):
        """
        @Xh: historical prices until time t, m*(t-1)
        Return:
        @portfolio: array of weights put on each assets
        """
        # Use h instead of t here since h = t-1
        cdef int numAssets = Xh.shape[0]
        cdef int h = Xh.shape[1]
        cdef list indexSet
        self.portfolio = np.ones(numAssets, dtype=np.float32)/numAssets
        # if have reached w(timeWindow) + 1 days
        if h > self.timeWindow:
            indexSet = [np.corrcoef(Xh[:,-self.timeWindow:].reshape(-1), Xh[:,i-self.timeWindow: i].reshape(-1))[1][0] >= self.rho\
                        for i in range(self.timeWindow ,h)]
            if sum(indexSet) > 0:
                self.portfolio = searchOpt(Xh[:,self.timeWindow:][:,indexSet])
        return self.portfolio
    def update(self, np.ndarray[float, ndim=1] xt):
        """
        Update at the end of the day
        @xt: prices of the current trading day
        """
        self.wealth *= np.sum(self.portfolio * xt)



def combine(np.ndarray[float, ndim=2] portfolioSet, np.ndarray[float, ndim=1] wealthSet,np.ndarray[float, ndim=1] q):
    """
    Combine the experts' portfolios
    @portfolioSet: array of portfolios, W * m(numAssets)
    @wealthSet: array of experts' current wealth, W
    @q: array of probability distribution function, m
    """
    cdef np.ndarray[float, ndim=1] nome
    cdef float deno
    nome = np.sum(portfolioSet.T * wealthSet * q, axis = 1)
    deno = np.sum(wealthSet * q)
    return nome/deno


def checkParams(**kwargs):
    if "rho" in kwargs:
        return False
    elif not "P" in kwargs and "K" in kwargs:
        raise ValueError("Definition for both P and K are required for CORNK")
    return True


def CORN(np.ndarray[float, ndim=2] X,int W, **kwargs):
    """
    main algorithm
    @W: maximum time window length
    @X: historical prices matrix, m(numAssets) * T
    There are two set of possible optional parameters:
    CORNU:
    @rho: correlation threshold
    CORNK:
    @P: max number of correlation thresholds
    @K: the value of K for top-K
    """
    logger = logging.getLogger('CORN')
    cdef bint IS_CORNK = checkParams(**kwargs)
    # Initialize weights q and wealth
    cdef float wealth = 1
    cdef list wealthRecord = [1.0]
    cdef int numAssets = X.shape[0]
    cdef int T = X.shape[1]
    
    cdef np.ndarray[float, ndim = 1] q,rhoSet, portfolioOverall
    cdef np.ndarray[long long, ndim=1] topK
    cdef float rho
    cdef int K,P,t
    cdef list expertPort, wealthSet
    cdef list portfolioSet
    cdef expert expertI
    if IS_CORNK:
        K = kwargs["K"]
        P = kwargs["P"]
        q = np.ones(W*P, dtype=np.float32)/(W*P)
        rhoSet = np.arange(P, dtype = np.float32)/P
        expertPort = [expert(rho,w,1) for w in range(1,W+1) for rho in rhoSet]
    else:
        q = np.ones(W)/W
        expertPort = [expert(rho,w,1) for w in range(1,W+1)]

    for t in range(T):
        if t%100 == 1:
            print(t)
        logger.info('Start Day ' + str(t))
        portfolioSet = [expertI.learning(X[:,:t]) for expertI in expertPort]      
        wealthSet = [expertI.wealth for expertI in expertPort]
        portfolioOverall = combine(np.array(portfolioSet), np.array(wealthSet).astype(np.float32), q)
        wealth *= np.sum(portfolioOverall * X[:,t])
        for expertI in expertPort:
            expertI.update(X[:,t])
        logger.info('End Day with wealth '+ str(wealth))
        wealthRecord.append(wealth)
        if IS_CORNK:
            topK = np.argsort(expertPort)[:K]
            logger.debug("The top K experts are: " +  str(topK))
            q.fill(0)
            q[topK] = 1/K
    return wealthRecord


def main():
    start = time.time()
    cdef np.ndarray[float, ndim=2] X = pkl.load(open("nyse_o_np.pkl", "rb"))[:,:500].astype(np.float32)
    cdef int W = 6
    cdef int P=5
    cdef int K=5
    cdef float rho = 0.4
    cdef list wealthRecord
    wealthRecord = CORN(X, W, P = P, K = K)
    print("total time: ", time.time() - start)

main()

1


C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\numpy\lib\function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Resurgam\Anaconda3\envs\advancedpy\lib\site-packages\numpy\lib\function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


101
201
301
401
total time:  786.8698189258575


In [124]:
main()

1
101
201
301
401
total time:  841.475870847702


# Numba - Correlation Matrix Computing

In [86]:
def computeIndex(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    indexSet = [np.corrcoef(Xh[:,-timeWindow:].reshape(-1), Xh[:,i-timeWindow: i].reshape(-1))[1][0] >= rho\
                        for i in range(timeWindow ,h)]
    return indexSet

In [68]:
X = pkl.load(open("nyse_o_np.pkl", "rb"))[:,:500]
%timeit -n100 result = computeIndex(X, 5, 0.5)

30.1 ms ± 260 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
from scipy.stats import pearsonr
def computeIndex2(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    indexSet = [pearsonr(Xh[:,-timeWindow:].reshape(-1), Xh[:,i-timeWindow: i].reshape(-1))[0] >= rho\
                        for i in range(timeWindow ,h)]
    return indexSet

In [25]:
%timeit -n100 computeIndex2(X, 5, 0.5)

23.1 ms ± 511 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [130]:
from numba import vectorize,jit,float64, int64

In [56]:
@jit
def computeIndex_vec(numAssets, h, timeWindow, rho, vec):
    #numAssets, h = Xh.shape
    #vec = Xh.reshape(-1, order = "F")
    fixed = vec[-timeWindow * numAssets:]
    indexSet =[]
    for i in range(timeWindow * numAssets, h * numAssets, numAssets):
        if pearsonr(fixed, vec[i-timeWindow* numAssets: i])[0] >= rho:
            indexSet.append(i)
    return indexSet


In [57]:
def computeIndex3(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    vec = Xh.reshape(-1, order = "F")
    return computeIndex_vec(numAssets, h, timeWindow,rho, vec)

In [59]:
%timeit -n100 computeIndex3(X, 5, 0.5)

19.9 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [115]:
@jit(nopython = True)
def computeIndex_vec2(numAssets, h, timeWindow, rho, vec):
    fixed = vec[-timeWindow * numAssets:]
    fixed_residual =fixed- np.mean(fixed)
    fixed_std = np.std(fixed)
    indexSet =[]
    rho_fake = rho * fixed_std * timeWindow * numAssets
    for i in range(timeWindow * numAssets, h * numAssets, numAssets):
        if np.sum(fixed_residual * (vec[i-timeWindow* numAssets: i] - np.mean(vec[i-timeWindow* numAssets: i])))/np.std(vec[i-timeWindow* numAssets: i]) >= rho_fake:
            indexSet.append(i//numAssets - timeWindow)
    return indexSet

In [116]:
@jit(nopython = True)
def computeIndex4(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    vec = np.ascontiguousarray(Xh.T).reshape(-1)
    return computeIndex_vec2(numAssets, h, timeWindow,rho, vec)

In [119]:
%timeit -n100 computeIndex4(X, 5, 0.5)

1.05 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [165]:
@jit("(float64[:,:], int64, float64)",nopython = True)
def computeIndex5(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    vec = np.ascontiguousarray(Xh.T).reshape(-1)
    fixed = vec[-timeWindow * numAssets:]
    fixed_residual =fixed- np.mean(fixed)
    fixed_std = np.std(fixed)
    indexSet =[]
    rho_fake = rho * fixed_std * timeWindow * numAssets
    for i in range(timeWindow * numAssets, h * numAssets, numAssets):
        if np.std(vec[i-timeWindow* numAssets: i]) > 0 and np.sum(fixed_residual * (vec[i-timeWindow* numAssets: i] - np.mean(vec[i-timeWindow* numAssets: i])))/np.std(vec[i-timeWindow* numAssets: i]) >= rho_fake:
            indexSet.append(i//numAssets - timeWindow)
    return indexSet

In [134]:
%timeit -n100 computeIndex5(X, 5, 0.5)

1.08 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
computeIndex4(X, 2, 0.4) == np.arange(498)[computeIndex(X, 2, 0.4)]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [166]:
computeIndex5(X[:,:2], 1, 0.5)

[]

In [163]:
def computeIndex6(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    vec = np.ascontiguousarray(Xh.T).reshape(-1)
    fixed = vec[-timeWindow * numAssets:]
    fixed_residual =fixed- np.mean(fixed)
    fixed_std = np.std(fixed)
    indexSet =[]
    rho_fake = rho * fixed_std * timeWindow * numAssets
    for i in range(timeWindow * numAssets, h * numAssets, numAssets):
        print( vec[i-timeWindow* numAssets: i] - np.mean(vec[i-timeWindow* numAssets: i]))

        if np.std(vec[i-timeWindow* numAssets: i]) > 0 and np.sum(fixed_residual * (vec[i-timeWindow* numAssets: i] - np.mean(vec[i-timeWindow* numAssets: i])))/np.std(vec[i-timeWindow* numAssets: i]) >= rho_fake:
            indexSet.append(i//numAssets - timeWindow)
    return indexSet

In [168]:
import sys
import numpy as np
from itertools import takewhile
import logging
from logbuilder import buildLogger
import cvxpy as cp
from numba import jit,int64,float64

def searchOpt(C):
    """
    @C: historical prices with corr high than rho
    """
    numAssets = C.shape[0]
    logger = logging.getLogger('CORN') 
    
    b = cp.Variable(numAssets, pos = True)
    prob = cp.Problem(cp.Minimize(-1 * cp.sum(cp.log(C.T * b))), [cp.sum(b) == 1])
    logger.debug('Solving Problem: ' + str(prob.solve(solver = 'SCS', warm_start = True)))
    return np.array(b.value)


class expert():
    def __init__(self, rho, timeWindow, initialWealth = 1):
        """
        initialize:
        @rho: correlation threshold
        @timeWindow: specific time windows
        @initialWealth: wealth
        """
        self.rho =rho
        self.timeWindow = timeWindow
        self.wealth = initialWealth

    def __eq__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth == other.wealth

    def __lt__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth < other.wealth
    
    @staticmethod
    @jit("(float64[:,:], int64, float64)",nopython = True)
    def computeIndex5(Xh, timeWindow, rho):
        numAssets, h = Xh.shape
        vec = np.ascontiguousarray(Xh.T).reshape(-1)
        fixed = vec[-timeWindow * numAssets:]
        fixed_residual =fixed- np.mean(fixed)
        fixed_std = np.std(fixed)
        indexSet =[]
        rho_fake = rho * fixed_std * timeWindow * numAssets
        for i in range(timeWindow * numAssets, h * numAssets, numAssets):
            if np.std(vec[i-timeWindow* numAssets: i]) > 0 and np.sum(fixed_residual * (vec[i-timeWindow* numAssets: i] - np.mean(vec[i-timeWindow* numAssets: i])))/np.std(vec[i-timeWindow* numAssets: i]) >= rho_fake:
                indexSet.append(i//numAssets - timeWindow)
        return indexSet

    def learning(self, Xh):
        """
        @Xh: historical prices until time t, m*(t-1)
        Return:
        @portfolio: array of weights put on each assets
        """
        # Use h instead of t here since h = t-1
        numAssets, h = Xh.shape
        self.portfolio = np.ones(numAssets)/numAssets
        # if have reached w(timeWindow) + 1 days
        if h > self.timeWindow:
            indexSet = computeIndex5(Xh, self.timeWindow, self.rho)
            if indexSet:
                self.portfolio = searchOpt(Xh[:,self.timeWindow:][:,indexSet])
        return self.portfolio
    def update(self, xt):
        """
        Update at the end of the day
        @xt: prices of the current trading day
        """
        self.wealth *= np.sum(self.portfolio * xt)


def combine(portfolioSet, wealthSet, q):
    """
    Combine the experts' portfolios
    @portfolioSet: array of portfolios, W * m(numAssets)
    @wealthSet: array of experts' current wealth, W
    @q: array of probability distribution function, m
    """
    nome = np.sum(portfolioSet.T * wealthSet * q, axis = 1)
    deno = np.sum(wealthSet * q)
    return nome/deno


def checkParams(**kwargs):
    if "rho" in kwargs:
        return False
    elif not "P" in kwargs and "K" in kwargs:
        raise ValueError("Definition for both P and K are required for CORNK")
    return True


def CORN(X, W, **kwargs):
    """
    main algorithm
    @W: maximum time window length
    @X: historical prices matrix, m(numAssets) * T
    There are two set of possible optional parameters:
    CORNU:
    @rho: correlation threshold
    CORNK:
    @P: max number of correlation thresholds
    @K: the value of K for top-K
    """
    logger = logging.getLogger('CORN')
    IS_CORNK = checkParams(**kwargs)
    # Initialize weights q and wealth
    wealth = 1
    wealthRecord = [1]
    numAssets, T = X.shape
    if IS_CORNK:
        K = kwargs["K"]
        P = kwargs["P"]
        q = np.ones(W*P)/(W*P)
        rhoSet = np.arange(P)/P
        expertPort = [expert(rho,w) for w in range(1,W+1) for rho in rhoSet]
    else:
        rho = kwargs["rho"]
        q = np.ones(W)/W
        expertPort = [expert(rho,w) for w in range(1,W+1)]

    for t in range(T):
        if t%100 == 1:
            print(t)
        logger.info('Start Day ' + str(t))
        portfolioSet = [expertI.learning(X[:,:t]) for expertI in expertPort]
        wealthSet = [expertI.wealth for expertI in expertPort]
        portfolioOverall = combine(np.array(portfolioSet), np.array(wealthSet), q)
        wealth *= np.sum(portfolioOverall * X[:,t])
        for expertI in expertPort:
            expertI.update(X[:,t])
        logger.info('End Day with wealth '+ str(wealth))
        wealthRecord.append(wealth)
        if IS_CORNK:
            topK = np.argsort(expertPort)[:K]
            logger.debug("The top K experts are: " +  str(topK))
            q.fill(0)
            q[topK] = 1/K
    return wealthRecord


def main():
    X = pkl.load(open("nyse_o_np.pkl", "rb"))[:,:500]
    W = 6
    P=5
    K=5
    start = time.time()
    wealthRecord = CORN(X, W, P = P, K = K)
    print("total time: ", time.time() - start)

main()

1
101
201
301
401
total time:  482.44442486763


1
101
201
301
401
total time:  488.308265209198


# Parallel Programming

In [18]:
%%writefile CORNU_test.py
from mpi4py import MPI
import numpy as np
from itertools import takewhile
import logging
import pickle as pkl
import time
from numba import jit,int64,float64
from logbuilder import buildLogger
import cvxpy as cp

import sys

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if not sys.warnoptions:
    import os,warnings
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

def searchOpt(C):
    """
    @C: historical prices with corr high than rho
    """
    numAssets = C.shape[0]
    b = cp.Variable(numAssets, pos = True)
    prob = cp.Problem(cp.Minimize(-1 * cp.sum(cp.log(C.T * b))), [cp.sum(b) == 1])
    prob.solve(solver = 'SCS', normalize= False)
    return np.array(b.value)

@jit("(float64[:,:], int64, float64)",nopython = True)
def computeIndex5(Xh, timeWindow, rho):
    numAssets, h = Xh.shape
    vec = np.ascontiguousarray(Xh.T).reshape(-1)
    fixed = vec[-timeWindow * numAssets:]
    fixed_residual =fixed- np.mean(fixed)
    fixed_std = np.std(fixed)
    indexSet =[]
    rho_fake = rho * fixed_std * timeWindow * numAssets
    for i in range(timeWindow * numAssets, h * numAssets, numAssets):
        if np.std(vec[i-timeWindow* numAssets: i]) > 0 and np.sum(fixed_residual * (vec[i-timeWindow* numAssets: i] - np.mean(vec[i-timeWindow* numAssets: i])))/np.std(vec[i-timeWindow* numAssets: i]) >= rho_fake:
            indexSet.append(i//numAssets - timeWindow)
    return indexSet

class expert():
    def __init__(self, rho, timeWindow, initialWealth = 1):
        """
        initialize:
        @rho: correlation threshold
        @timeWindow: specific time windows
        @initialWealth: wealth
        """
        self.rho =rho
        self.timeWindow = timeWindow
        self.wealth = initialWealth

    def __eq__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth == other.wealth

    def __lt__(self, other):
        """
        Used for wealth ranking
        """
        return self.wealth < other.wealth
    
    #@staticmethod

    def learning(self, Xh):
        """
        @Xh: historical prices until time t, m*(t-1)
        Return:
        @portfolio: array of weights put on each assets
        """
        # Use h instead of t here since h = t-1
        numAssets, h = Xh.shape
        self.portfolio = np.ones(numAssets)/numAssets
        # if have reached w(timeWindow) + 1 days
        if h > self.timeWindow:
            indexSet = computeIndex5(Xh, self.timeWindow, self.rho)
            if indexSet:
                self.portfolio = searchOpt(Xh[:,self.timeWindow:][:,indexSet])
        return self.portfolio
    def update(self, xt):
        """
        Update at the end of the day
        @xt: prices of the current trading day
        """
        self.wealth *= np.sum(self.portfolio * xt)


def combine_comm(wealth_portfolio, q):
    """
    Combine the experts' portfolios, specially designed for mpi
    @wealth_portfolio: first column is wealth, the rest are portfolios.numExperts * T * (numAssets + 1) 
    @q: array of probability distribution function, m
    """
    combined_p = []
    for t in range(T):
        wealthSet = wealth_portfolio[:,t,0].reshape(-1)
        portfolioSet = wealth_portfolio[:,t,1:]
        nome = np.sum(portfolioSet.T * wealthSet * q, axis = 1)
        deno = np.sum(wealthSet * q)
        combined_p.append(nome/deno)  
    return combined_p



numAssets = 36
T = 500
W = 5
rho = 0.4
assert W == size

logger = logging.getLogger('CORN')

recvbuf = np.empty((size, T,numAssets + 1))
sendbuf = np.empty((T, numAssets + 1))

#rank 0 is designed to deal with ocmbining
if rank == 0:
    X = np.ascontiguousarray(pkl.load(open("nyse_o_np.pkl", "rb"))[:numAssets,:T])
    q = np.ones(W)/W
    wealth = 1
    wealthRecord = [1]
    start = time.time()
else: 
    X = np.empty((numAssets, T))
w = rank + 1
expertI = expert(rho,w)

#broadcasting
comm.Bcast(X, root = 0)

for t in range(T):
    if t%100 == 0:
        print("rank {} has reached {} days".format(rank,t))
    sendbuf[t,1:] = expertI.learning(X[:,:t])
    sendbuf[t,0] = expertI.wealth
    expertI.update(X[:,t])
    sendbuf = np.ascontiguousarray(sendbuf)
    
#gather portfolio and wealth
comm.Gather(sendbuf, recvbuf, root = 0)    
if rank == 0:
    #print(recvbuf[:,0,:])
    #pkl.dump(recvbuf,open('recvbuf', 'wb'))
    portfolioOverall = combine_comm(recvbuf, q)
    for t in range(T):
        wealth *= np.sum(portfolioOverall[t] * X[:,t]) 
        wealthRecord.append(wealth)
        
    print("total time: ", time.time() - start)


Overwriting CORNU_test.py


# Expensive Pooling

In [12]:
from cornu_multi import CORNU
start = time.time()
wr, pr = CORNU(0.4, 5, X)
end = time.time()

In [13]:
end-start

1737.327616930008

In [197]:
end-start

1714.3396577835083

# One-time Pooling

In [14]:
from couru_multi2 import CORNU
import time
start = time.time()
wr, pr = CORNU(0.4, 5, X)
end = time.time()

In [15]:
end-start

20.009546756744385